In [1]:
import os
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

In [2]:
!apt-get install -y libzbar0
!pip install pyzbar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-text libdjvulibre21 libfftw3-double3
  libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0 libjxr-tools libjxr0
  liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libv4l-0 libv4lconvert0 libwmflite-0.2-7 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x libfftw3-bin libfftw3-dev inkscape poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-text libdjvulibre21 libfftw3-

In [3]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.0 MB/s eta 0:00:00


In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
from ultralytics import YOLO
model = YOLO("yolov8n")
model = YOLO("/content/drive/MyDrive/1pharma/runs/detect/qr_detection3/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
base_dir = "/content/drive/MyDrive/1pharma"
os.makedirs(base_dir, exist_ok=True)

In [12]:
import cv2
import json
import os
import numpy as np
from pyzbar.pyzbar import decode
from PIL import Image
from google.colab import files

def process_image(image_path, model):
    # Use filename (without extension) as image_id
    image_id = os.path.splitext(os.path.basename(image_path))[0]

    # Read image
    img = cv2.imread(image_path)

    # YOLO inference
    results = model(img)  # returns ultralytics Results object
    detections = results[0].boxes.xyxy.cpu().numpy()  # [x_min, y_min, x_max, y_max]

    qrs = []
    for box in detections:
        x1, y1, x2, y2 = box[:4]

        # Crop QR region
        crop = img[int(y1):int(y2), int(x1):int(x2)]

        # Decode QR using pyzbar
        decoded_objects = decode(Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)))
        value = decoded_objects[0].data.decode("utf-8") if decoded_objects else "UNKNOWN"

        qrs.append({
            "bbox": [int(x1), int(y1), int(x2), int(y2)],
            "value": value
        })

    return {"image_id": image_id, "qrs": qrs}

def save_and_download_json(result, output_file):
    with open(output_file, "w") as f:
        json.dump([result], f, indent=2, separators=(",", ": "))
    print(f"✅ JSON saved to {output_file}")

    # Trigger download in Colab
    files.download(output_file)

# Example usage
image_path = "/content/drive/MyDrive/1pharma/Test_clr/images/img202.jpg"
result = process_image(image_path, model)

# output filename derived from image name
output_file = f"{os.path.splitext(os.path.basename(image_path))[0]}.json"
save_and_download_json(result, output_file)


0: 384x640 1 QR, 132.0ms
Speed: 5.5ms preprocess, 132.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
✅ JSON saved to img202.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>